# Notebook Script 08-slope-to-edges

This Notebook edits and tests Script 08

In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/home/jovyan/accesibilidad-urbana/aup/data.py:25: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(


## Script editing

In [6]:
schema = 'osmnx'
#table_sufix = 'elevation_24' #sufix for nodes and edges table names
grl_path = '../../../data/external/MDE/'
#main(schema, folder_sufix, year, grl_path, save=True)

metro_schema = 'metropolis'
metro_table = 'metro_gdf_2020' # Cities 2020

# --------------- SCRIPT
# Load cities (municipalities)
query = f"SELECT city FROM {metro_schema}.{metro_table}"
metro_df = aup.df_from_query(query)
city_list = list(metro_df.city.unique())
k = len(city_list)

In [7]:
city = 'Aguascalientes'

In [8]:
# Creates empty GeoDataFrame to store specified municipality polygons and hex grid
mun_gdf = gpd.GeoDataFrame()
# Iterates over municipality codes for each metropolitan area or capital
query = f"SELECT * FROM {metro_schema}.{metro_table} WHERE \"city\" LIKE \'{city}\'"
mun_gdf = aup.gdf_from_query(query, geometry_col='geometry')

#Define projections for municipalities and hexgrids
mun_gdf = mun_gdf.set_crs("EPSG:4326")

# Creates query to download OSMNX nodes and edges from the DB
# by metropolitan area or capital using the municipality geometry
G, nodes, edges = aup.graph_from_hippo(mun_gdf, schema='osmnx', edges_folder='edges_23_line', nodes_folder='nodes_23_point')
aup.log(f"Downloaded {len(nodes)} nodes and {len(edges)} edges from database for {city}")

In [44]:
# Show
print(nodes.shape)
nodes.head(1)

(55006, 5)


,x,y,street_count,geometry,city
osmid,,,,,
272921360,-102.295073,21.872876,3,POINT (-102.29507 21.87288),Aguascalientes


In [45]:
# Show
print(edges.shape)
edges.head(1)

(128284, 16)


,,,osmid,oneway,lanes,name,highway,maxspeed,length,geometry,bridge,ref,junction,tunnel,access,width,service,city
u,v,key,,,,,,,,,,,,,,,,
1417904191,4507389643,0,1036118103,False,None,Calle San Martín de la Cantera,unclassified,None,74.345,"LINESTRING (-102.35592 21.85416, -102.35540 21...",None,None,None,None,None,None,None,Aguascalientes


In [14]:
mde_path = [] # list to append mde path strings
#Gathers state codes for MDE
for e in mun_gdf.CVE_ENT.unique():
    tmp_path = grl_path + (f'CEM_V3_20170619_R15_E{e}_TIF/')

    #search for files in tmp_path for .tif
    for f in os.listdir(tmp_path):
        if f.endswith('.tif'):
            mde_path.append(tmp_path+f)

In [16]:
#elevations to nodes
G_elev_mde = ox.elevation.add_node_elevations_raster(G, mde_path)
#slope to edges
G_elev_mde = ox.elevation.add_edge_grades(G_elev_mde, add_absolute=True, precision=3)
nodes_elev_mde, edges_elev_mde = ox.graph_to_gdfs(G_elev_mde, nodes=True, edges=True)
mean_elev = round(nodes_elev_mde.elevation.mean(),2)
mean_slope = round(edges_elev_mde.grade_abs.mean(),2)
aup.log(f"Assigned a mean elevation of {mean_elev} to nodes "+
f"\nand mean slope of {mean_slope} to edges")
# reset index for upload
nodes_elev_mde.reset_index(inplace=True)
edges_elev_mde.reset_index(inplace=True)

#set street_count as float
nodes_elev_mde["street_count"] = nodes_elev_mde["street_count"].astype(float)

/tmp/ipykernel_90156/1505067729.py:4: UserWarning: the `precision` parameter is deprecated and will be removed in a future release
  G_elev_mde = ox.elevation.add_edge_grades(G_elev_mde, add_absolute=True, precision=3)


In [27]:
edges_elev_mde.head(1)

,u,v,key,osmid,oneway,name,highway,length,geometry,city,...,grade_abs,lanes,bridge,maxspeed,ref,access,junction,service,tunnel,width
0,1417904191,4507389643,0,1036118103,False,Calle San Martín de la Cantera,unclassified,74.345,"LINESTRING (-102.35592 21.85416, -102.35540 21...",Aguascalientes,...,0.027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
nodes_elev_mde.head(1)

,osmid,x,y,street_count,city,elevation,geometry
0,1417904191,-102.355922,21.85416,3.0,Aguascalientes,1867.0,POINT (-102.35592 21.85416)


## Test (After running Aguascalientes on Script 08 on Visual) 

In [2]:
# Load test
saved_schema = 'osmnx'
saved_nodes_table = 'nodes_elevation_23_point'
saved_edges_table = 'edges_elevation_23_line'

In [3]:
query = f"SELECT * FROM {saved_schema}.{saved_nodes_table}"
nodes_test = aup.gdf_from_query(query)

# Show
print(nodes_test.shape)
nodes_test.head(1)

(55006, 6)


,osmid,x,y,street_count,elevation,geometry
0,1417904191,-102.355922,21.85416,3.0,1867.0,POINT (-102.35592 21.85416)


In [4]:
query = f"SELECT * FROM {saved_schema}.{saved_edges_table}"
edges_test = aup.gdf_from_query(query)

# Show
print(edges_test.shape)
edges_test.head(1)

(128284, 20)


,u,v,key,osmid,oneway,name,highway,length,geometry,grade,grade_abs,lanes,bridge,maxspeed,ref,access,junction,service,tunnel,width
0,1417904191,4507389643,0,1036118103,False,Calle San Martín de la Cantera,unclassified,74.345,"LINESTRING (-102.35592 21.85416, -102.35540 21...",0.027,0.027,None,None,None,None,None,None,None,None,None
